<a href="https://colab.research.google.com/github/Mehrnoushhk/DeepLearning/blob/master/PinAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime
import mplfinance as mpl

In [ ]:
%%time
import DataCleaner
from DataCleaner import dataCleaner
#url= 'https://raw.githubusercontent.com/Mehrnoushhk/DeepLearning/master/GBPUSDm30.csv'
#data= pd.read_csv(url) 
data= pd.read_csv('C:\\Users\\Maryam-Goli\\Documents\\GitHub\\DeepLearning\\GBPUSDm30.csv', skiprows= 28000)
data= dataCleaner(data)

In [ ]:
%%time
from DataCleaner import localMin
from DataCleaner import localMax
data= localMin(data, 4)
data= localMax(data, 4)

In [ ]:
minMax= data[['minValue', 'maxValue']]
adp= mpl.make_addplot(minMax[len(minMax)-200:], type= 'scatter', markersize= 200)
mpl.plot(data.iloc[data.shape[0]-200: data.shape[0],:], type= 'candle', style= 'charles', figscale= 6, addplot= adp, axisoff= True)

In [ ]:
columnsNames= data.columns
data = data.reindex(columns = data.columns.tolist() + ['Trend', 'TrendValue'])

In [ ]:
%%time
from DataCleaner import nextTrend
data= nextTrend(data)

In [ ]:
adp= mpl.make_addplot(data.iloc[data.shape[0]-200:, 11], type= 'scatter', markersize= 200)
mpl.plot(data.iloc[data.shape[0]-200: data.shape[0],:], type= 'candle', style= 'charles', figscale=6, addplot= adp, axisoff= True)

In [ ]:
data.loc[data['Trend'] == -1].count()

In [ ]:
%%time
j= data.columns.get_loc('Trend')
correction_counter= 1
uptrend_counter= 1
downtrend_counter= 1
for k in range (200, data.shape[0], 100):
    for i in range(100, k):
        if data.iloc[i,j] == 1:
            upname= 'C:\\Users\\Maryam-Goli\\Documents\\CNNGBP\\UpTrend\\'+'Uptrend'+ str(uptrend_counter)+ '.png'
            mpl.plot(data.iloc[i-100:i, :], type= 'candle', style= 'charles', axisoff= True, savefig= upname)
            uptrend_counter = uptrend_counter+ 1
        elif data.iloc[i,j] == 0:
            corname= 'C:\\Users\\Maryam-Goli\\Documents\\CNNGBP\\Consolidation\\'+'Consolidation'+ str(correction_counter)+ '.png'
            mpl.plot(data.iloc[i-100:i, :], type= 'candle', style= 'charles', axisoff= True, savefig= corname)
            correction_counter= correction_counter+ 1
        elif data.iloc[i, j] == -1:
            downname= 'C:\\Users\\Maryam-Goli\\Documents\\CNNGBP\\DownTrend\\'+'Downtrend'+ str(downtrend_counter)+ '.png'
            mpl.plot(data.iloc[i-100:i, :], type= 'candle', style= 'charles', axisoff= True, savefig= downname)
            downtrend_counter = downtrend_counter+ 1
 

In [ ]:
rcparams = {'axes.spines.bottom':False,
            'axes.spines.left':False,
            'axes.spines.right':False,
            'axes.spines.top':False,
            'axisoff': True
            }
myStyle= mpl.make_mpf_style(base_mpf_style= 'charles', gridcolor= 'white', rc= rcparams)
mpl.plot(upTrend.iloc[0:100,:], type= 'candle', style= myStyle)

In [ ]:
data['Uptrend']= np.NaN
j= data.columns.get_loc("Trend")
k= data.columns.get_loc('Uptrend')
l= data.columns.get_loc('Low')
for i in range(0, data.shape[0]):
    if data.iloc[i, j] == 1:
        data.iloc[i, k] = data.iloc[i, l]

for i in range(80, 300):
    if data.iloc[i, j] == 1:
         adp= mpl.make_addplot(data.iloc[i-80:i+10, k], type= 'scatter', markersize= 200)
         mpl.plot(data.iloc[i-80:i+10, :], type= 'candle', style= 'charles', figscale= 4, addplot= adp)

In [ ]:
counter= 1
for i in range(100, 200):
    if data.iloc[i, trendid]== 1:
        mpl.plot(data.iloc[i-100:i, :], type= 'candle', title= str(counter))
        mpl.plot(data.iloc[i: i+10,:], type= 'candle', title= str(counter))
        counter= counter+ 1

In [ ]:
pip install --upgrade mplfinance